# Code Executors
Code executors enable agents to not just chat but also to interact with an environment and perform useful computations and take actions.

## Overview
| Code Executor (autogen.coding)        | Environment         | Platform      |
|---------------------------------------|---------------------|---------------|
| [LocalCommandLineCodeExecutor](#)    | Shell               | Local         |
| [DockerCommandLineCodeExecutor](#)   | Shell               | Docker        |
| [jupyter.JupyterCodeExecutor](#)     | Jupyter Kernel (e.g., python3) | Local/Docker |

## Docker Execution
To mitigate the security risk of running LLM-generated code locally, we can use the docker command line code executor (`autogen.coding.DockerCommandLineCodeExecutor`) to execute code in a docker container. This way, the generated code can only access resources that are explicitly given to it.

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

config_list = [
  {
    "model": os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    "api_type": "azure",
    "api_key": os.environ['AZURE_OPENAI_API_KEY'],
    "base_url": os.environ['AZURE_OPENAI_ENDPOINT'],
    "api_version": os.environ['AZURE_OPENAI_API_VERSION']
  }
]

In [ ]:
import tempfile

from autogen import ConversableAgent
from autogen.coding import DockerCommandLineCodeExecutor

# Create a temporary directory to store the code files.
temp_dir = tempfile.TemporaryDirectory()

# Create a Docker command line code executor.
executor = DockerCommandLineCodeExecutor(
    image="python:3.12-slim",  # Execute code using the given docker image name.
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir="tmp3uclztv5",  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration that uses docker.
code_executor_agent_using_docker = ConversableAgent(
    "code_executor_agent_docker",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the docker command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

# When the code executor is no longer used, stop it to release the resources.
# executor.stop()

/home/vscode/.local/lib/python3.12/site-packages/flaml/__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


In [8]:
# The code writer agent's system message is to instruct the LLM on how to use
# the code executor in the code executor agent.
code_writer_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when everything is done.
"""

code_writer_agent = ConversableAgent(
    "code_writer_agent",
    system_message=code_writer_system_message,
    llm_config={"config_list": config_list},
    code_execution_config=False,  # Turn off code execution for this agent.
)

In [5]:
chat_result = code_executor_agent_using_docker.initiate_chat(
    code_writer_agent,
    message="Write Python code to calculate the 14th Fibonacci number.",
)

code_executor_agent_docker (to code_writer_agent):

Write Python code to calculate the 14th Fibonacci number.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_writer_agent (to code_executor_agent_docker):

Sure, I'll provide you with Python code to calculate the 14th Fibonacci number:

```python
# filename: fibonacci_14.py

def fibonacci(n):
    if n <= 0:
        return "Invalid Input"
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n):
            a, b = b, a + b
        return b

# Calculate the 14th Fibonacci number
fib_14 = fibonacci(14)
print(f"The 14th Fibonacci number is: {fib_14}")
```

To execute this code, save it in a file named `fibonacci_14.py` and run it. The program will calculate and print the 14th Fibonacci number.

--------------------------------------------------------------------------------


In [9]:
import datetime

today = datetime.datetime.now().strftime("%Y-%m-%d")
chat_result = code_executor_agent_using_docker.initiate_chat(
    code_writer_agent,
    message=f"Today is {today}. Write Python code to plot TSLA's and META's "
    "stock price gains YTD, and save the plot to a file named 'stock_gains.png'.",
)

code_executor_agent_docker (to code_writer_agent):

Today is 2025-01-03. Write Python code to plot TSLA's and META's stock price gains YTD, and save the plot to a file named 'stock_gains.png'.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_writer_agent (to code_executor_agent_docker):

To solve this task, we will:

1. Import necessary libraries.
2. Fetch stock price data for TSLA and META from a financial data API.
3. Calculate the year-to-date (YTD) stock price gains.
4. Plot the stock price gains.
5. Save the plot to a file named 'stock_gains.png'.

Here's the necessary Python code to accomplish these steps:

```python
# filename: plot_stock_gains.py

import yfinance as yf
import matplotlib.pyplot as plt
import datetime

# Define the date range
start_date = "2025-01-01"
end_date = "2025-01-03"

# Fetch the stock data
tsla_data = yf.download("TSLA", start=start_date, end=end_date)
meta_data = yf.download("META", sta